In [1]:
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
import pandas as pd

In [4]:
import sklearn as sk

In [5]:
sk.__version__

'1.3.0'

In [6]:
dataset = pd.read_csv('diabetes_prediction_dataset.csv')

In [7]:
dataset

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [8]:
X = dataset.iloc[:, 0:9].values
##Column 0 is Gender Column, Negative 1 is the ID##
y = dataset.iloc[:, -1].values

In [9]:
X

array([['Female', 80.0, 0, ..., 6.6, 140, 0],
       ['Female', 54.0, 0, ..., 6.6, 80, 0],
       ['Male', 28.0, 0, ..., 5.7, 158, 0],
       ...,
       ['Male', 66.0, 0, ..., 5.7, 155, 0],
       ['Female', 24.0, 0, ..., 4.0, 100, 0],
       ['Female', 57.0, 0, ..., 6.6, 90, 0]], dtype=object)

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 0] = le.fit_transform(X[:, 0])
##0 Is Female , 1 is Male##

In [11]:
print(X)

[[0 80.0 0 ... 6.6 140 0]
 [0 54.0 0 ... 6.6 80 0]
 [1 28.0 0 ... 5.7 158 0]
 ...
 [1 66.0 0 ... 5.7 155 0]
 [0 24.0 0 ... 4.0 100 0]
 [0 57.0 0 ... 6.6 90 0]]


In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
## Hot Encoding Smoke History# 

In [13]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [4])], remainder='passthrough')

In [14]:
X = np.array(ct.fit_transform(X))

In [15]:
print(X)

[[0.0 0.0 0.0 ... 6.6 140 0]
 [1.0 0.0 0.0 ... 6.6 80 0]
 [0.0 0.0 0.0 ... 5.7 158 0]
 ...
 [0.0 0.0 0.0 ... 5.7 155 0]
 [0.0 0.0 0.0 ... 4.0 100 0]
 [0.0 1.0 0.0 ... 6.6 90 0]]


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
sc = StandardScaler()

In [20]:
X_train = sc.fit_transform(X_train)

In [21]:
X_test = sc.transform(X_test)

In [22]:
#ANN#

In [23]:
ann = tf.keras.models.Sequential()

In [24]:
ann.add(tf.keras.layers.Dense(units=4, activation='relu'))

In [25]:
ann.add(tf.keras.layers.Dense(units=4, activation='relu'))

In [26]:
ann.add(tf.keras.layers.Dense(units=4, activation='selu'))

In [27]:
ann.add(tf.keras.layers.Dense(units=4, activation='softsign'))

In [29]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Tried Using Softplus optimizer, horrible accuracy tbh#

In [30]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 50)

Epoch 1/50
1875/1875 [==============================] - 3s 1ms/step - loss: 0.6762 - accuracy: 0.0639
Epoch 2/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3587 - accuracy: 0.0040
Epoch 3/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3470 - accuracy: 0.0046
Epoch 4/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3419 - accuracy: 0.0918
Epoch 5/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3387 - accuracy: 0.3378
Epoch 6/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3365 - accuracy: 0.6747
Epoch 7/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3349 - accuracy: 0.8378
Epoch 8/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3338 - accuracy: 0.9111
Epoch 9/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3330 - accuracy: 0.9144
Epoch 10/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.332

In [31]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

1250/1250 [==============================] - 1s 912us/step


In [32]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

ValueError: cannot reshape array of size 160000 into shape (40000,1)

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [34]:
import matplotlib.pyplot as plt